In [5]:
import pandas as pd
import numpy as np

from shapely.geometry import Point, shape, mapping
from shapely.ops import unary_union

import matplotlib.pyplot as plt

#from geoband.API import *
import folium
import json
import geopandas as gpd
from geopandas import gpd

from folium import plugins
import re
from haversine import haversine

In [6]:
bike = pd.read_csv("finaldata/bike.csv",encoding = "cp949")
bus = pd.read_csv("finaldata/bus.csv",encoding = "cp949")
data = pd.read_csv("finaldata/data.csv",encoding = "cp949")
park = pd.read_csv("finaldata/park.csv",encoding = "cp949")
parking = pd.read_csv("finaldata/parking.csv",encoding = "cp949")
pop = pd.read_csv("finaldata/population.csv",encoding = "cp949")
river = pd.read_csv("finaldata/river.csv",encoding = "cp949")
road = pd.read_csv("finaldata/road.csv",encoding = "cp949")
subway = pd.read_csv("finaldata/subway.csv",encoding = "cp949")
traf = pd.read_csv("finaldata/traffic.csv",encoding = "cp949")
tree = pd.read_csv("finaldata/tree.csv",encoding = "cp949")
vent = pd.read_csv("finaldata/vent.csv",encoding = "cp949")

/Users/seungji/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,5,6,8,12,13,15,17,18,20,25,31,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data

,serial,lat,lon,초미세먼지(㎍/㎥),미세먼지(㎍/㎥),기온(℃),상대습도( %),초미세먼지 보정(㎍/㎥),미세먼지 보정(㎍/㎥)
0,V02Q1940046,37.494403,126.831472,52.687500,82.208333,15.192361,45.857639,39.513889,88.312500
1,V02Q1940111,37.509299,126.878380,36.372483,47.540268,16.516779,42.315436,32.382550,65.053691
2,V02Q1940165,37.496892,126.843727,32.639610,42.948052,14.998052,46.321429,29.512987,59.087662
3,V02Q1940056,37.487430,126.838527,35.226950,46.343972,14.177305,48.251773,32.510638,64.904255
4,V02Q1940047,37.482145,126.889339,51.609442,77.467811,15.512446,44.381974,38.051502,95.935622
...,...,...,...,...,...,...,...,...,...
1055,OC3CL200044,37.560409,126.978669,28.027119,75.223729,14.387797,41.216949,27.037288,85.471186
1056,OC3CL200212,37.566048,126.977400,25.141935,69.258065,14.041290,42.458065,19.541935,56.964516
1057,OC3CL200202,37.594270,127.074112,27.270175,70.877193,13.761053,40.901754,24.452632,80.361404
1058,OC3CL200203,37.600165,127.111919,26.489796,77.387755,13.025510,48.816327,25.363946,80.846939


In [4]:
lyon = (45.7597, 4.8422) # (lat, lon)
paris = (48.8567, 2.3508)

haversine(lyon, paris)

392.2172595594006

### 2) 교통수단
#### 2-1 버스정류장

버스정류장은 보통 300-500m 거리 간격이라고 한다.    
센서에서 100m 이내에 있는 버스정류장을 데이터에 넣어준다. 

In [5]:
bus

,표준ID,정류소명,경도,위도,승차총승객수,하차총승객수
0,100000001,종로2가사거리,126.987786,37.569764,104.866667,94.758333
1,100000002,창경궁.서울대학교병원,126.996520,37.579179,126.936364,139.298485
2,100000003,명륜3가.성대입구,126.998290,37.582709,168.845098,132.668627
3,100000004,종로2가.삼일교,126.987507,37.568582,63.753731,68.332090
4,100000005,혜화동로터리.여운형활동터,127.001694,37.586230,85.456250,131.483333
...,...,...,...,...,...,...
10475,124900125,강동리버스트4단지,127.170805,37.573357,79.677419,4.032258
10476,124900126,강동리버스트4단지,127.171136,37.572897,79.677419,4.032258
10477,124900108,암사삼성.현대아파트,127.128600,37.553307,50.833333,42.100000
10478,124000334,우성아파트,127.139326,37.550442,134.314815,117.955556


geodataframe으로 바꿔서 data 를 buffer한 후 bus 포인트가 몇개 카운트 되는지 세기

In [7]:
data_g = gpd.GeoDataFrame(data, geometry = gpd.points_from_xy(data.lon,data.lat),crs={'init': 'epsg:4326'})

/Users/seungji/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [8]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(300,resolution=7))
bus_g = gpd.GeoDataFrame(bus, geometry = gpd.points_from_xy(bus.경도,bus.위도),crs={'init': 'epsg:4326'})
bus_g1 = bus_g.to_crs('epsg:3395')

In [8]:
cnt=[]
for i in range(len(data_g1)):
    cnt.append(bus_g1.within(data_g1.geometry[i]))

In [9]:
버스승차승객수 = []
버스하차승객수 = []
for c in cnt:
    버스승차승객수.append(bus_g1.loc[c,:].승차총승객수.mean())
    버스하차승객수.append(bus_g1.loc[c,:].하차총승객수.mean())

In [10]:
data1 = pd.concat([data,pd.DataFrame({"버스승차승객수":버스승차승객수,"버스하차승객수":버스하차승객수})],axis=1)
# 300m 안에 버스정류장이 없는 경우 최솟값으로 na imputation
data1.loc[:,"버스승차승객수"] = data1.버스승차승객수.fillna(data1.버스승차승객수.min())
data1.loc[:,"버스하차승객수"] = data1.버스하차승객수.fillna(data1.버스하차승객수.min())

gdf = bus_g1.to_crs(epsg='4737')
converted_json = gdf.to_json()
gdf1 = data_g1.to_crs(epsg='4737')
converted_json1 = gdf1.to_json()

m = folium.Map(
    location=[37.54238169591348,126.99068745821812],
 attr = '서울',
    zoom_start=12
)

folium.GeoJson(
    converted_json
).add_to(m)

folium.GeoJson(
    converted_json1
).add_to(m)

m

In [ ]:
gpd.overlay(data_g1,bus_g,how="intersection")

In [ ]:
geom9 = gpd.points_from_xy(data9['보호구역_경도'], data9['보호구역_위도'])
gpd9 = gpd.GeoDataFrame(geometry = geom9, crs = from_epsg(4326))
crs9 = gpd9.to_crs(epsg = '4326')
buf9 = gpd.GeoDataFrame()
buf9.geometry = crs9.buffer(0.0005)
poly9 = buf9.to_crs(epsg = '4326')
over9 = gpd.overlay(data2, poly9, how = 'intersection')
over9

#### 2-2 자전거 대여정보

현재 따릉이 대여소 간 평균 거리는 500m이다. 2020년엔 대여소가 확대되어 대여소 간 거리는 300m로 줄어든다. 

In [18]:
bike

,대여소 번호,보관소(대여소)명,위도,경도,대여,반납
0,301,경복궁역 7번출구 앞,37.575794,126.971451,413,333
1,302,경복궁역 4번출구 뒤,37.575947,126.974060,711,721
2,303,광화문역 1번출구 앞,37.571770,126.974663,587,590
3,304,광화문역 2번출구 앞,37.572113,126.977577,391,405
4,305,종로구청 옆,37.572582,126.978355,508,542
...,...,...,...,...,...,...
2001,1085,래미안 솔베뉴(고덕로),37.554569,127.145874,173,132
2002,1086,래미안 솔베뉴(구천면로),37.550598,127.145454,141,140
2003,1088,풍납사거리(강동),37.528122,127.119522,212,218
2004,1089,고덕 래미안힐스테이트(201동),37.560600,127.146698,52,53


In [19]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(300,resolution=7))
bike_g = gpd.GeoDataFrame(bike, geometry = gpd.points_from_xy(bike.경도,bike.위도),crs={'init': 'epsg:4326'})
bike_g1 = bike_g.to_crs('epsg:3395')

/Users/seungji/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [20]:
cnt=[]
for i in range(len(data_g1)):
    cnt.append(bike_g1.within(data_g1.geometry[i]))

In [21]:
대여 = []
반납 = []
for c in cnt:
    대여.append(bike_g1.loc[c,:].대여.mean())
    반납.append(bike_g1.loc[c,:].반납.mean())

In [24]:
data2 = pd.concat([data1,pd.DataFrame({"대여":대여,"반납":반납})],axis=1)
# 300m 안에 버스정류장이 없는 경우 최솟값으로 na imputation
data2.loc[:,"대여"] = data2.대여.fillna(data2.대여.min())
data2.loc[:,"반납"] = data2.반납.fillna(data2.반납.min())

#### 2-3 교통혼잡도

In [56]:
traf

,지점,traf_mean,지점번호,위도,경도,geometry
0,성산로(금화터널),1229.838889,A-01,37.568588,126.948436,"POLYGON ((14133335.253 4492604.897, 14133297.6..."
1,사직로(사직터널),1352.365278,A-02,37.572298,126.962853,"POLYGON ((14134940.146 4493123.762, 14134902.5..."
2,자하문로(자하문터널),570.393750,A-03,37.588831,126.968548,"POLYGON ((14135574.110 4495436.312, 14135536.5..."
3,대사관로(삼청터널),238.295139,A-04,37.596359,126.984209,"POLYGON ((14137317.485 4496489.461, 14137279.8..."
4,율곡로(안국역),1059.229861,A-05,37.576000,126.984342,"POLYGON ((14137332.290 4493641.533, 14137294.6..."
...,...,...,...,...,...,...
128,북부간선로,2222.276389,F-04,37.608556,127.052577,"POLYGON ((14144928.176 4498196.020, 14144890.5..."
129,동부간선도로,2830.603028,F-05,37.568685,127.076023,"POLYGON ((14147538.172 4492618.463, 14147500.5..."
130,경부고속도로,4018.057078,F-06,37.493135,127.022533,"POLYGON ((14141583.693 4482058.020, 14141546.0..."
131,분당수서로,2789.965384,F-07,37.497648,127.087195,"POLYGON ((14148781.834 4482688.548, 14148744.2..."


In [57]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(500,resolution=7))
traf_g = gpd.GeoDataFrame(traf, geometry = gpd.points_from_xy(traf.경도,traf.위도),crs={'init': 'epsg:4326'})
traf_g = traf_g.to_crs('epsg:3395')

/Users/seungji/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [58]:
cnt=[]
for i in range(len(data_g1)):
    cnt.append(traf_g.within(data_g1.geometry[i]))

In [59]:
traf_mean = []
for c in cnt:
    traf_mean.append(traf_g.loc[c,:].traf_mean.mean())

In [60]:
data3 = pd.concat([data2,pd.DataFrame({"traffic":traf_mean})],axis=1)
# 500m 안에 도로가 없는 경우 최솟값으로 na imputation
data3.loc[:,"traffic"] = data3.traffic.fillna(data3.traffic.min()*0.8)

#### 2-4 자동차 전용도로

In [63]:
road

,시군구코드,위도,경도
0,11545,37.464545,126.901719
1,11680,37.519419,127.031031
2,11680,37.516691,127.028994
3,11680,37.524409,127.056466
4,11140,37.554729,127.007670
...,...,...,...
48561,11530,37.504453,126.855258
48562,11200,37.545618,127.046348
48563,11305,37.643109,127.012256
48564,11215,37.532349,127.073581


In [79]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(300,resolution=7))
road_g = gpd.GeoDataFrame(road, geometry = gpd.points_from_xy(road.경도,road.위도),crs={'init': 'epsg:4326'})
road_g = road_g.to_crs('epsg:3395')

In [80]:
cnt=[]
for i in range(len(data_g)):
    cnt.append(road_g.within(data_g1.geometry[i]))

In [81]:
road_mean = pd.DataFrame(cnt).transpose().sum()

In [82]:
data3 = pd.concat([data3,pd.DataFrame({"road":road_mean})],axis=1)

#### 2-5 주차장

In [75]:
parking

,주차장코드,주차장명,주소,주차장 종류명,위도,경도
0,1010089,초안산근린공원주차장(구),도봉구 창동 24-0,노외 주차장,37.649279,127.040474
1,1012254,마들스타디움(근린공원)(구),노원구 상계동 770-2,노외 주차장,37.643917,127.058567
2,1013181,마장동공영주차장(구),성동구 마장동 463-2,노외 주차장,37.569939,127.035145
3,1025695,영등포여고 공영(구),영등포구 신길동 184-3,노외 주차장,37.515205,126.915116
4,1025696,당산근린공원 공영(구),영등포구 당산동3가 385-0,노외 주차장,37.525526,126.895794
...,...,...,...,...,...,...
14402,968503,삼양마을공원지하 공영주차장(구),강북구 미아동 748-2,노외 주차장,37.622745,127.018428
14403,968514,색동공원 공영주차장(구),강북구 수유동 49-7,노외 주차장,37.633227,127.020711
14404,980509,비봉주차장(구),종로구 구기동 139-9,노외 주차장,37.612180,126.956535
14405,984617,신도림역환승주차장(구),구로구 구로동 1-4,노외 주차장,37.507334,126.891653


In [189]:
parking1 = parking.loc[parking["주차장 종류명"]=="노상 주차장",:].groupby(by = "주차장명").mean().reset_index(drop=False)
parking2 = parking.loc[parking["주차장 종류명"]=="노외 주차장",["주차장명","주차장코드","위도","경도"]]

In [199]:
parking = pd.concat([parking1,parking2]).reset_index(drop=False).iloc[:,1:]

In [202]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(300,resolution=7))
parking_g = gpd.GeoDataFrame(parking, geometry = gpd.points_from_xy(parking.경도,parking.위도),crs={'init': 'epsg:4326'})
parking_g = parking_g.to_crs('epsg:3395')

In [203]:
cnt=[]
for i in range(len(data_g)):
    cnt.append(parking_g.within(data_g1.geometry[i]))

In [204]:
parking_mean = pd.DataFrame(cnt).transpose().sum()

In [205]:
data3 = pd.concat([data3,pd.DataFrame({"parking":parking_mean})],axis=1)

In [209]:
data3.columns

Index(['serial', 'lat', 'lon', '초미세먼지(㎍/㎥)', '미세먼지(㎍/㎥)', '기온(℃)', '상대습도( %)',
       '초미세먼지 보정(㎍/㎥)', '미세먼지 보정(㎍/㎥)', 'geometry', '버스승차승객수', '버스하차승객수', '대여',
       '반납', 'traffic', 'road', 'road', 'parking', 'subway', 'vent',
       'parking'],
      dtype='object')

data3 = data3.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,18,19,20]]

#### 2-6 지하철 정보

In [88]:
subway

,line,name,code,lat,lng
0,01호선,녹양,1908.0,37.759380,127.042292
1,01호선,남영,1002.0,37.541021,126.971300
2,01호선,용산,1003.0,37.529849,126.964561
3,01호선,노량진,1004.0,37.514219,126.942454
4,01호선,대방,1005.0,37.513342,126.926382
...,...,...,...,...,...
692,인천선,지식정보단지,3135.0,37.378384,126.645168
693,인천선,인천대입구,3136.0,37.386007,126.639484
694,인천선,센트럴파크,3137.0,37.393054,126.634729
695,인천선,국제업무지구,3138.0,37.399907,126.630347


In [97]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(300,resolution=7))
subway_g = gpd.GeoDataFrame(subway, geometry = gpd.points_from_xy(subway.lng,subway.lat),crs={'init': 'epsg:4326'})
subway_g = subway_g.to_crs('epsg:3395')

In [98]:
cnt=[]
for i in range(len(data_g)):
    cnt.append(subway_g.within(data_g1.geometry[i]))

In [99]:
subway_mean = pd.DataFrame(cnt).transpose().sum()

In [103]:
subway_mean

0        33
1        57
2        37
3        33
4        45
       ... 
1055    102
1056    102
1057     74
1058     38
1059     42
Length: 1060, dtype: int64

In [143]:
data3 = pd.concat([data3,pd.DataFrame({"subway":subway_mean})],axis=1)

data3.to_csv("data/data3_0617.csv",index=False, encoding = "cp949")

In [ ]:
gdf = data_g1.to_crs(epsg='4737')
converted_json = gdf.to_json()

m = folium.Map(
    location=[37.54238169591348,126.99068745821812],
 attr = '서울',
    zoom_start=12
)

folium.GeoJson(
    converted_json
).add_to(m)
m

#### 2-7 환기구 정보

In [113]:
vent = gpd.read_file("data/UQ152_도시계획시설_교통시설(도로외)/uq152.shp")
#facility = facility.set_crs('epsg:4737',allow_override=True)

In [114]:
vent_tf = []
for i in range(len(vent)):
    vent_tf.append("환기구" in vent.DGM_NM[i])

In [115]:
vent = vent.loc[vent_tf,["DGM_NM","SHAPE_AREA","geometry"]]

In [116]:
vent

,DGM_NM,SHAPE_AREA,geometry
149,5호선 환기구,176.879745,"POLYGON ((193442.011 447015.609, 193446.498 44..."
150,5호선 환기구,313.030649,"POLYGON ((183090.492 452607.873, 183120.423 45..."
151,6호선 본선환기구,159.243638,"POLYGON ((191357.655 451706.836, 191360.566 45..."
152,분당선 환기구,439.496794,"POLYGON ((207251.538 443897.129, 207226.449 44..."
153,환기구,423.405148,"MULTIPOLYGON (((203095.586 454358.541, 203091...."
...,...,...,...
1849,6호선 환기구,210.733589,"POLYGON ((195639.158 449421.302, 195644.908 44..."
1850,6호선 환기구,284.669143,"POLYGON ((207764.818 457711.701, 207735.804 45..."
1851,7호선 환기구,139.804295,"POLYGON ((204913.939 446713.696, 204913.752 44..."
1852,7호선 환기구,203.509818,"POLYGON ((199530.185 443160.405, 199530.412 44..."


In [131]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(100,resolution=7))
vent_g = gpd.GeoDataFrame(vent, geometry =vent.geometry.buffer(100),crs={'init': 'epsg:4326'})
vent_g = vent_g.to_crs('epsg:3395')

/Users/seungji/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
<ipython-input-131-0ad598309275>:3: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  vent_g = gpd.GeoDataFrame(vent, geometry =vent.geometry.buffer(100),crs={'init': 'epsg:4326'})


In [132]:
vent_g.shape

(279, 3)

In [133]:
data_g1.shape

(1060, 10)

In [134]:
vent_g1 = gpd.overlay(data_g1,vent_g,how="intersection")

In [140]:
vent_g1.serial.nunique()

1051

In [146]:
vent_g2 =pd.DataFrame(vent_g1.serial.value_counts()).reset_index()
vent_g2.columns = ["serial","vent"]

In [151]:
data3 = data3.merge(vent_g2, how = "left",left_on = "serial",right_on = "serial")

In [155]:
data3["vent"] = data3.vent.fillna(0)

data3.to_csv("data/data3.csv",index=False,encoding="cp949")

In [9]:
data3 = pd.read_csv("data/data3.csv",encoding="cp949")

### 3. 인구
#### 3-1 행정구역별 인구
#### 3-2 거주인구
#### 3-3 인구이동
#### 3-4 인구밀도

In [10]:
pop = pd.read_csv("finaldata/population.csv",encoding = "cp949")

In [11]:
pop = pop.drop([330,357])
pop = pop.reset_index(drop=True)

In [12]:
seoul = gpd.read_file("data/LARD_ADM_SECT_SGG_서울/LARD_ADM_SECT_SGG_11.shp",encoding = "cp949")
dong = gpd.read_file("data/HangJeongDong_ver20210401.geojson.txt",delimiter = "\t")
dong = dong.loc[dong.sidonm=="서울특별시",:]
dong = dong.reset_index(drop=True)

In [13]:
temp = []
for i in dong.adm_nm:
    temp.append(i.split()[1]+i.split()[2])

temp = pd.DataFrame(temp).replace(["종로구종로1·2·3·4가동","종로구종로5·6가동","성동구금호2·3가동","중랑구면목3·8동","노원구상계3·4동","노원구상계6·7동","노원구중계2·3동"],["종로구종로1.2.3.4가동","종로구종로5.6가동","성동구금호2.3가동","중랑구면목3.8동","노원구상계3.4동","노원구상계6.7동","노원구중계2.3동"])

In [14]:
temp1 = temp[0]

In [15]:
temp1.columns = "구동"

In [16]:
check1 = []
for i in list(temp1):
    if i in list(pop.구동):check1.append(False)
    else: check1.append(True)

In [17]:
temp[check1]

,0
270,구로구항동
287,영등포구당산2동
320,관악구행운동


In [18]:
dong = pd.concat([dong,temp1],axis=1)

In [19]:
dong = dong[["geometry",0]]

In [20]:
dong.columns = ["geometry","구동"]

In [21]:
pop1 = dong.merge(pop,how = "left",left_on = "구동", right_on = "구동")

In [22]:
for i in ['총생활인구수', '남10대생활인구수', '여10대생활인구수', '남20대생활인구수', '여20대생활인구수',
       '남30,40대생활인구수', '여30,40대생활인구수', '남50,60대생활인구수', '여50,60대생활인구수',
       '남자70세이상생활인구수', '여자70세이상생활인구수', '세대', '인구', '세대당인구',
       '외국인인구비율', '구내전입', '구내전출', '구간전입', '구간전출', '인구밀도']:
    pop1.loc[pop1.구동 == "종로구종로5.6가동",i] = pop1.loc[pop1.구동.isin(["종로구창신1동","종로구창신2동","종로구이화동","중구신당동","중구광희동","중구을지로동","종로구종로1.2.3.4가동"]),i].mean()
    pop1.loc[pop1.구동 == "구로구항동",i] = pop1.loc[pop1.구동.isin(["구로구오류2동","구로구수궁동"]),i].mean()
    pop1.loc[pop1.구동 == "영등포구당산2동",i] = pop1.loc[pop1.구동.isin(["영등포구양평2동","영등포구양평1동","영등포구당산1동","영등포구영등포동"]),i].mean()
    pop1.loc[pop1.구동 == "관악구행운동",i] = pop1.loc[pop1.구동.isin(["동작구사당5동","관악구청림동","관악구중앙동","관악구청룡동","관악구인헌동","동작구사당4동"]),i].mean()

In [23]:
for i in range(len(pop1)):
    pop1.loc[i,"위도"] = pop1.loc[i,"geometry"].centroid.y
    pop1.loc[i,"경도"] = pop1.loc[i,"geometry"].centroid.x

In [24]:
data4 = data3

In [25]:
data4 = gpd.GeoDataFrame(data4,geometry = gpd.points_from_xy(data4.lon,data4.lat),crs={'init': 'epsg:4326'})

/Users/seungji/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [26]:
cnt=[]
for i in range(len(pop1)):
    cnt.append(data4.within(pop1.geometry[i]))
    

In [27]:
temp = pd.DataFrame(cnt)

In [28]:
구동t = []
for i in range(len(data4)):
    if temp[i].sum() == 0: 구동t.append('')
    else: 구동t.append(list(pop1.구동[temp[i]])[0])

In [29]:
data5 = pd.concat([data4,pd.DataFrame({"구동":구동t})],axis=1)

In [30]:
data6 =data5.merge(pop1,how = "left",left_on = "구동",right_on = "구동")

In [31]:
data6.columns

Index(['serial', 'lat', 'lon', '초미세먼지(㎍/㎥)', '미세먼지(㎍/㎥)', '기온(℃)', '상대습도( %)',
       '초미세먼지 보정(㎍/㎥)', '미세먼지 보정(㎍/㎥)', 'geometry_x', '버스승차승객수', '버스하차승객수',
       '대여', '반납', 'traffic', 'road', 'subway', 'vent', 'parking', '구동',
       'geometry_y', '통계청행정동코드', '행자부행정동코드', '시군구명', '행정동명', '경도', '위도',
       '총생활인구수', '남10대생활인구수', '여10대생활인구수', '남20대생활인구수', '여20대생활인구수',
       '남30,40대생활인구수', '여30,40대생활인구수', '남50,60대생활인구수', '여50,60대생활인구수',
       '남자70세이상생활인구수', '여자70세이상생활인구수', '자치구', '동', '세대', '인구', '세대당인구',
       '외국인인구비율', '구내전입', '구내전출', '구간전입', '구간전출', '인구밀도'],
      dtype='object')

In [71]:
data6.to_csv("data/data6.csv",encoding="cp949")

### 4. 자연시설물
#### 4-1 공원

park = pd.read_csv("finaldata/park.csv",encoding = "cp949")

In [108]:
park

,공원번호,공원명,면적,경도,위도,geometry
0,1,남산도시자연공원,2896887.0,126.990377,37.550140,POINT (126.99038 37.55014)
1,3,길동생태공원,80683.0,127.154779,37.540394,POINT (127.15478 37.54039)
2,4,서울대공원,9132690.0,127.019846,37.426449,POINT (127.01985 37.42645)
3,5,서울숲,480994.0,127.041798,37.543072,POINT (127.04180 37.54307)
4,6,월드컵공원,2284085.0,126.878907,37.571805,POINT (126.87891 37.57180)
...,...,...,...,...,...,...
127,1378,용마도시자연공원(사가정공원),109635.0,127.095997,37.579762,POINT (127.09600 37.57976)
128,1379,문화비축기지,140022.0,126.893246,37.571718,POINT (126.89325 37.57172)
129,1380,경춘선숲길,211392.0,127.076482,37.627077,POINT (127.07648 37.62708)
130,1381,율현공원,157535.0,127.115594,37.472332,POINT (127.11559 37.47233)


In [109]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(1000,resolution=7))
park_g = gpd.GeoDataFrame(park, geometry = gpd.points_from_xy(park.경도,park.위도),crs={'init': 'epsg:4326'})
park_g = park_g.to_crs('epsg:3395')

/Users/seungji/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [110]:
park_g1 = gpd.GeoDataFrame(park_g,geometry = park_g.geometry.buffer(500,resolution=7))

In [111]:
cnt=[]
for i in range(len(data_g1)):
    cnt.append(park_g1.within(data_g1.geometry[i]))

In [112]:
park_mean = pd.DataFrame(cnt).transpose().sum()

In [116]:
park_mean.value_counts()

1    391
2    273
0    175
3    132
4     61
5     18
6      7
7      3
dtype: int64

In [117]:
park_gg = pd.DataFrame(park_mean)

In [119]:
park_gg.to_csv("data/park_gg.csv",index = False,encoding="cp949")

In [115]:
data6 = pd.concat([data6,pd.DataFrame({"park":park_mean})],axis=1)

In [64]:
gdf = park_g1.to_crs(epsg='4737')
converted_json = gdf.to_json()

m = folium.Map(
    location=[37.54238169591348,126.99068745821812],
 attr = '서울',
    zoom_start=12
)

folium.GeoJson(
    converted_json
).add_to(m)
m

#### 4-2 가로수

In [73]:
tree = gpd.read_file("data/TN_STTREE_W.shp")

tree = gpd.read_file("data/TN_STTREE_W.shp")
tree_g = pd.concat([tree.GU_NM,tree.geometry.x,tree.geometry.y],axis=1)
tree_g = tree_g.rename(columns = {0:"경도",1:"위도"})

In [82]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(100,resolution=7))

In [85]:
tree = tree.to_crs('epsg:3395')

In [87]:
cnt=[]
for i in range(len(data_g)):
    cnt.append(tree.within(data_g1.geometry[i]))

In [88]:
tree_mean = pd.DataFrame(cnt).transpose().sum()

tree_mean = []
for c in cnt:
    tree_mean.append(tree.loc[c,:].tree_mean.mean())

In [90]:
data7 = pd.concat([data6,pd.DataFrame({"tree":tree_mean})],axis=1)
# 500m 안에 도로가 없는 경우 최솟값으로 na imputation
#data3.loc[:,"tree"] = data3.traffic.fillna(data3.traffic.min()*0.8)

In [91]:
data7.to_csv("data/data7.csv",encoding="cp949",index = False)

#### 4-3 하천호수

In [92]:
river

,시군구코드,하천/호수명_한글,위도,경도
0,11260,,37.611528,127.073543
1,11305,,37.640491,127.007029
2,11740,,37.552968,127.146748
3,11740,,37.523360,127.144726
4,11110,,37.572876,126.994545
...,...,...,...,...
608,11710,,37.503313,127.146743
609,11170,,37.540697,127.006598
610,11680,,37.512457,127.039686
611,11410,,37.571550,126.947281


In [93]:
data_g1 = data_g.to_crs('epsg:3395')
data_g1 = gpd.GeoDataFrame(data_g,geometry = data_g1.geometry.buffer(500,resolution=7))
river_g = gpd.GeoDataFrame(river, geometry = gpd.points_from_xy(river.경도,river.위도),crs={'init': 'epsg:4326'})
river_g = river_g.to_crs('epsg:3395')

/Users/seungji/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [95]:
cnt=[]
for i in range(len(data_g)):
    cnt.append(river_g.within(data_g1.geometry[i]))

In [96]:
river_mean = pd.DataFrame(cnt).transpose().sum()

In [ ]:
river_mean = []
for c in cnt:
    river_mean.append(river_g.loc[c,:].river_mean.mean())

In [98]:
data8 = pd.concat([data7,pd.DataFrame({"river":river_mean})],axis=1)

In [99]:
data8.to_csv("data/finaldata0617.csv",encoding="cp949",index = False)